# **Random Forest**

Log-loss Score obtained: **0.69800**

## **Setup**

In [1]:
!git clone https://github.com/yeray142/AIML_project

Cloning into 'AIML_project'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 53 (delta 17), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [2]:
cd AIML_project

/content/AIML_project


In [12]:
import pandas as pd
import numpy as np

train = pd.read_csv("data/cleaned_train.csv", index_col=0)
test = pd.read_csv("data/cleaned_test.csv", index_col=0)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

## **Data preprocessing**

In [14]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,No job,2.0,-6.0,1.0
1,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0,0.0
2,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0,2.0
3,F,N,Y,2,270000.0,Working,Secondary / secondary special,Married,House / apartment,-13413,-4996,0,0,1,High skill tech staff,4.0,-18.0,1.0
4,F,N,N,0,315000.0,Working,Secondary / secondary special,Separated,House / apartment,-17570,-1978,0,0,1,Core staff,1.0,-41.0,2.0


In [15]:
test.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month
index,,,,,,,,,,,,,,,,,
26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,21990,0,0,1,0,No job,2.0,60
26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,18964,8671,0,1,0,Core staff,2.0,36
26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,15887,217,1,1,0,Laborers,2.0,40
26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,19270,2531,1,0,0,Drivers,2.0,41
26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,17822,9385,1,0,0,Managers,2.0,8


In [16]:
cat = ["income_type", "edu_type", "family_type", "house_type", "occup_type", "gender", "car", "reality"]

for x in cat:
  lenc = LabelEncoder()
  lenc = lenc.fit(train[x])
  train[x] = lenc.transform(train[x])
  test[x] = lenc.transform(test[x])

In [20]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month,credit
0,0,0,0,0,202500.0,0,1,1,2,13899,4709,0,0,0,12,2.0,6,1.0
1,0,0,1,0,202500.0,0,4,1,1,15088,2092,0,1,0,15,2.0,37,0.0
2,0,1,1,0,157500.0,2,1,1,1,15037,2105,0,0,0,10,2.0,26,2.0
3,0,0,1,2,270000.0,4,4,1,1,13413,4996,0,0,1,6,4.0,18,1.0
4,0,0,0,0,315000.0,4,4,2,1,17570,1978,0,0,1,3,1.0,41,2.0


In [18]:
test.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month
index,,,,,,,,,,,,,,,,,
26457,1,1,0,0,112500.0,1,4,0,1,21990,0,0,1,0,12,2.0,60
26458,0,0,1,0,135000.0,2,1,1,1,18964,8671,0,1,0,3,2.0,36
26459,0,0,1,0,69372.0,4,4,1,1,15887,217,1,1,0,8,2.0,40
26460,1,1,0,0,112500.0,0,4,1,1,19270,2531,1,0,0,4,2.0,41
26461,0,1,1,0,225000.0,2,1,1,1,17822,9385,1,0,0,10,2.0,8


## **Model and train**

In [101]:
def stratified_kfold_rf(p, n_fold, X, y, X_test):
    # Declaring Stratified K-Fold:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    
    # Initializing arrays:
    rf_oof = np.zeros((X.shape[0], 3))
    rf_preds = np.zeros((X_test.shape[0], 3))

    # Main loop:
    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============")
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # Model fit:
        model = RandomForestClassifier(**p)
        model.fit(X_train, y_train)

        rf_oof[valid_idx] = model.predict_proba(X_valid) # Infers the class probability of train dataset
        rf_preds += model.predict_proba(X_test) / n_fold # Infers the class probability of test dataset
        print(f"Log Loss Score: {log_loss(y_valid, rf_oof[valid_idx]):.5f}")

    log_score = log_loss(y, rf_oof)
    print(f"\nLog Loss Score: {log_score:.5f}")

    return rf_oof, rf_preds

In [96]:
p = {
  "criterion": "entropy",
  "n_estimators": 300,
  "min_samples_split": 10,
  "min_samples_leaf": 2,
  "max_features": "sqrt",
  "oob_score": True,
  "random_state": 42,
  "n_jobs": -1,
  }

In [97]:
X = train.drop("credit", axis=1)
y = train["credit"]
X_test = test.copy()

In [102]:
rf_oof, rf_preds = stratified_kfold_rf(p, 10, X, y, X_test)

============ Fold 0 ============
Log Loss Score: 0.69977
============ Fold 1 ============
Log Loss Score: 0.70089
============ Fold 2 ============
Log Loss Score: 0.69099
============ Fold 3 ============
Log Loss Score: 0.68497
============ Fold 4 ============
Log Loss Score: 0.69733
============ Fold 5 ============
Log Loss Score: 0.69550
============ Fold 6 ============
Log Loss Score: 0.71542
============ Fold 7 ============
Log Loss Score: 0.69163
============ Fold 8 ============
Log Loss Score: 0.69932
============ Fold 9 ============
Log Loss Score: 0.70417

Log Loss Score: 0.69800


In [47]:
rf_preds.shape

(10000, 3)

In [103]:
rf_oof.shape

(23398, 3)

## **Save results**

In [82]:
submission = pd.read_csv("data/sample_submission.csv")

In [83]:
submission.set_index("index", inplace=True)

In [84]:
submission[:] = rf_preds

In [85]:
submission.head()

,0,1,2
index,,,
26457,0.088317,0.151100,0.760582
26458,0.234536,0.202485,0.562979
26459,0.057679,0.090341,0.851980
26460,0.094220,0.107161,0.798619
26461,0.091143,0.173934,0.734923
